# Obtener un promedio de los 3 días anteriores
Pronóstico (WRF 1.5) ajustado con climatología mensual (observada) menos observado (Estaciones Meteorológicas IMN)

## Cargar Librerias
Se cargan las librerías esenciales para la manipulación de datos, visualización, cálculos numéricos, manejo de fechas, interacción con la web y operaciones del sistema operativo.

In [1]:
import glob  # Para buscar archivos en un directorio.
import pandas as pd  # Para manipulación de datos.
import numpy as np  # Para cálculos numéricos.
from datetime import datetime, date, timedelta  # Importamos clases relacionadas con fechas y horas.
import calendar  # para trabajar con calendarios.
import requests  # Para realizar solicitudes HTTP.
from tabulate import tabulate  # Para crear tablas formateadas.
import os  # Para interactuar con el sistema operativo.

## Carga de datos observados de los 3 días anteriores
Se utiliza un diccionario llamado "meses_espanol" para asociar números del 1 al 12 con los nombres de los meses en español. Luego, obtiene la fecha actual y calcula las fechas de ayer, anteayer y el día antes de anteayer.

Define una URL base (`enlace_base`) para acceder a archivos. Se crea un diccionario vacío llamado `datos_observados` para almacenar el contenido de archivos descargados y utiliza un bucle para iterar sobre una lista de tuplas que contienen fechas anteriores y nombres de archivos correspondientes. Dentro del bucle, extrae el año, mes y día de cada fecha, y el nombre del mes en español. Luego, construye una URL para el archivo utilizando la información de la fecha.

A través de solicitudes HTTP, si la respuesta indica que el archivo existe (código de estado HTTP 200) descarga el contenido de cada archivo de texto que contienen registros de datos correspondientes a esas fechas. Si el archivo no existe, se muestra un mensaje de error.

In [15]:
# Diccionario que asocia números de meses con sus nombres en español
meses_espanol = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo", 6: "junio", 7: "julio",
    8: "agosto", 9: "setiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# URL base del servidor para acceder a los archivos diarios de temperatura observada
enlace_base = "http://intra-files.imn.ac.cr/Intranet_graficos/datos5minutos/tablas_registro"

datos_observados = {} # Diccionario para almacenar el contenido de los archivos descargados
file_date = [] # Lista para almacenar las fechas de los archivos descargados
missing_data = [] # Lista para almacenar las fechas de archivos faltantes

# Lista de fechas de los últimos 3 días
fechas = [date.today() - timedelta(days=i) for i in range(1,4)]

# Recorrer las fechas y descargar archivos de temperatura observada correspondientes
for fecha in fechas:
    # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
    año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]
    # Crear el URL completo del archivo de temperatura observada utilizando los componentes de la fecha
    url_archivo = f"{enlace_base}/{año}/{mes_text}/temperaturas_{dia}_{mes}_{año}.txt"

    # Verificar si el archivo de temperatura observada existe antes de descargarlo
    response = requests.get(url_archivo)

    if response.status_code == 200: #Archivo existe: código de estado HTTP 200
        datos_observados[fecha] = response.text #Agregar el archivo correspondiente a la fecha en datos_observados
        file_date.append(fecha.strftime("%Y%m%d")) # Agregar la fecha en formato YYYYMMDD a file_date
    else:
        #Imprimir un mensaje indicando que para la fecha indicada el archivo de temperatura observada correspondiente no existe
        print(f"El archivo de la {fecha} no existe en la URL: {url_archivo}")
        missing_data.append(fecha.strftime("%Y%m%d")) # Agregar la fecha del archivo faltante en formato YYYYMMDD
        
        #Verificar que no falten más de dos archivos de temperatura observada
        if len(missing_data) < 2:
            # En caso de que el archivo no esté disponible, obtener datos de temperatura observada de cuatro días atrás
            fecha = date.today() - timedelta(days=4) # Obtener la nueva fecha
            # Desglosar la fecha en sus componentes: año, mes, día y nombre del mes en español
            año, mes, dia, mes_text = fecha.year, fecha.month, fecha.day, meses_espanol[fecha.month]
            # Crear el URL completo del archivo de temperatura observada utilizando los componentes de la fecha
            url_archivo = f"{enlace_base}/{año}/{mes_text}/{dia}_{mes}_{año}_acum024.txt"

            # Verificar si el archivo existe antes de descargarlo
            response = requests.get(url_archivo)

            if response.status_code == 200:#Archivo existe: código de estado HTTP 200
                datos_observados[fecha] = response.text #Agregar el archivo correspondiente a la fecha en datos_observados
                file_date.append(fecha.strftime("%Y%m%d")) # Agregar la fecha en formato YYYYMMDD
            else:
                #Imprimir un mensaje indicando que para la fecha indicada el archivo de temperatura observada no existe
                print(f"El archivo de la {fecha} tampoco existe en la URL: {url_archivo}")
        else:
            # Si faltan más de dos archivos de temperatura observada, imprimir un mensaje.
            print('Faltan más de dos archivos')
            
datos_observados = dict(sorted(datos_observados.items())) # Ordenar el diccionario que contiene los datos de temperatura observada por fecha
file_date.sort()# Ordenar la lista de fechas de los archivos de temperatura observada

## Procesamiento y Almacenamiento de Datos
En primer lugar, se crea una lista vacía llamada `obs`, esta lista se utilizará para almacenar los DataFrames resultantes de los datos observados. Luego, utiliza un bucle para iterar a través de las parejas de `file_date` (fechas en formato texto) y el contenido de archivos previamente guardados en `datos_observados`.

Dentro del bucle, el contenido del archivo se divide en líneas utilizando `"\r\n"` como separador. Se inicia una lista llamada `estaciones` para almacenar la información de las estaciones meteorológicas que se extraerá del archivo. Además, se establece una variable `seccion_encontrada` para rastrear si se ha encontrado la sección relevante en el archivo.

Luego, el código procede a procesar cada línea del archivo. Si una línea contiene `"*****"` o `"....."`, esto indica que se ha encontrado la sección deseada. Si `seccion_encontrada` es `True` y la línea contiene un guión `"-"`, esto indica que contiene datos relevantes para una estación. Se dividen los datos utilizando espacios como separadores y se realiza una limpieza de datos, eliminando espacios en blanco y elementos vacíos. Si los datos extraídos no están vacíos y no comienzan con `"-----"`, se extraen la estación, la temperatura máxima y mínima.

Se comprueba si los valores de tmax y tmin no son "NA" y, si son válidos, se convierten en tipo float64. El valor de estación se convierte en tipo entero. Los datos de estación válidos se agregan a la lista `estaciones` como tuplas (estacion, tmax, tmin).

Después de procesar todas las líneas del archivo, la lista `estaciones` se convierte en un DataFrame de Pandas llamado `df_estaciones` con columnas "Estacion", "Tmax" y "Tmin". Luego, se genera una ruta de carpeta `carpeta` donde se guardarán los archivos resultantes.

Finalmente, el DataFrame `df_estaciones` se agrega a la lista `obs`.

In [3]:
# Crear una lista para almacenar los DataFrames de datos de temperatura observada luego de ajustar el formato
obs = []

# Iterar sobre las fechas y DataFrames de lluvia observada
for date, archivo in zip(file_date, datos_observados.values()):
    # Dividir el contenido del archivo en líneas usando "\n" como separador
    lineas = archivo.strip().split("\n")[1:]  # Omitir la primera línea que es la cabecera

    # Inicializar una lista para almacenar los datos de las estaciones de este archivo
    estaciones = []

    # Variable para determinar si hemos encontrado la sección con estaciones
    seccion_encontrada = False

    # Procesar cada línea para extraer los datos de las estaciones
    for linea in lineas:
        if "*****" in linea:
            seccion_encontrada = True
        elif "....." in linea:
            seccion_encontrada = True
        elif seccion_encontrada and "-" in linea:
            datos_estacion = linea.split("   ")

            # Eliminar líneas vacías antes de procesar los datos
            datos_estacion = [dato.strip() for dato in datos_estacion if dato.strip()]

            # Asegurarnos de que la línea no esté vacía después de la eliminación
            if datos_estacion and not datos_estacion[0].startswith("-----"):
                estacion = datos_estacion[0].split("-")[0]
                tmax = datos_estacion[4]  # Corregir el índice para obtener la longitud
                tmin = datos_estacion[5]   # Corregir el índice para obtener la latitud
                
                # Solo usar las lineas donde tmax, tmin es diferente a 'NA'
                if tmax != "NA" and tmin != "NA":
                    tmax = float(tmax)# Convertir el valor a tipo float64
                    tmin = float(tmin)# Convertir el valor a tipo float64
                    estacion = int(estacion)# Convertir el valor a tipo int
                    
                    # Agregar los datos de la estación, tmax y tmin a la lista estaciones
                    estaciones.append((estacion, tmax, tmin))

    # Convertir la lista de estaciones en un DataFrame de Pandas para este archivo
    df_estaciones = pd.DataFrame(estaciones, columns=["Estacion", "Tmax", "Tmin"])

    # Agregar el DataFrame con el nuevo formato a la lista de DataFrames
    obs.append(df_estaciones)

## Cargar documentos del modelo de los 3 días anteriores

Cargar archivos CSV ubicados en un directorio especificado en una lista llamada `pro`. Utiliza un bucle `for` para iterar a través de una lista de fechas (Fechas de archivos de datos observados), construye las rutas completas a los archivos CSV, verifica su existencia y, si se encuentran, los carga en DataFrames de pandas que son agregados a la lista `pro`. Si un archivo no existe en la ubicación especificada, se muestra un mensaje de advertencia.

In [4]:
# Directorio donde se encuentran los archivos de temperatura pronosticada
directorio_modelo = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Mod_ajuste_w_Clim/Datos/Temperatura"

# Crear una lista para almacenar los DataFrames de temperatura pronosticada
pro = []

# Iterar sobre las fechas disponibles
for date in file_date:
    # Construir el URL del archivo de temperatura pronosticada para la fecha correspondiente
    url_file = f"{directorio_modelo}/{date}.temperatura.csv"

    # Verificar si el archivo de temperatura pronosticada existe en la ubicación
    if os.path.exists(url_file):
        # Leer el archivo CSV como un DataFrame de Pandas
        file = pd.read_csv(url_file)
        
        # Agregar el DataFrame a la lista que contendrá todos los datos temperatura pronosticada para los 3 días previos
        pro.append(file)
    else:
        # Imprimir un mensaje si el archivo de temperatura pronosticada no existe
        print(f"El archivo {url_file} no existe.")

## Pronostico menos Observado

Se crea una lista llamada `tablas_diferencias` para almacenar las tablas que contendrán las diferencias calculadas. Luego, se recorren las listas `pro` y `obs` en un bucle para calcular las diferencias entre los DataFrames correspondientes y guardar los resultados en archivos CSV.

Dentro del bucle, se realiza una combinación "left" entre los DataFrames `pro[i]` y `obs[i]` basada en la columna "Estacion". Esto combina los datos de ambas fuentes en un solo DataFrame llamado `diferencias`.

Se calculan las diferencias restando los valores de los DataFrames `obs` de los valores de `pro`, y se agregan estas diferencias como una nueva columna llamadas `Diferencia_Tmax` y `Diferencia_Tmin`, en el DataFrame `diferencias`.

Luego, se seleccionan las columnas necesarias ("Estacion", "Diferencia_Tmax" y "Diferencia_Tmin") del DataFrame `diferencias` y se guarda este DataFrame en la lista `tablas_diferencias`.

In [5]:
# Crear una lista para almacenar los resultados (lluvia pronosticada menos temperatura observada)
tablas_diferencias = []

# Recorrer las listas de DataFrames (pro y obs) para calcular las diferencias (pro - obs) para cada día
for i in range(len(pro)):
    # Realizar una combinación "left" entre pro y obs en base a la columna "Estacion"
    diferencias = pro[i].merge(obs[i], on="Estacion", how="left", suffixes=("_pro", "_obs"))

    # Calcular las diferencias (pro - obs) y guardarlo en la columna 'Diferencia_Tmax' y 'Diferencia_Tmin'
    diferencias["Diferencia_Tmax"] = diferencias["Tmax_pro"] - diferencias["Tmax_obs"]
    diferencias["Diferencia_Tmin"] = diferencias["Tmin_pro"] - diferencias["Tmin_obs"]

    # Seleccionar solo las columnas necesarias
    diferencias = diferencias[["Estacion", "Diferencia_Tmax", "Diferencia_Tmin"]]

    # Agregar la tabla diferencias a la lista tablas_diferencias
    tablas_diferencias.append(diferencias)

## Crear o Ajustar Tabla de Promedios

La función `crear_tabla_si_no_existe` recibe los parámetros: `carpeta`, que representa la ruta de la carpeta donde se guardará la tabla; `nombre_tabla1` y `nombre_tabla2`, que es el nombre de la tablas a crear.

Se verifica si la carpeta especificada existe utilizando la función `os.path.exists`. Si la carpeta no existe, se crea con `os.makedirs(carpeta)`. Esto garantiza que la carpeta esté disponible para almacenar la tabla. Se construye la ruta completa del archivo de la tabla combinando la ruta de la `carpeta` y el nombre de tabla con la extensión ".csv".

Se verifica si el archivo de la tabla ya existe en la ruta proporcionada. Si el archivo no existe, se crea una tabla vacía utilizando solo la columna "Estacion" del primer DataFrame `pro[0]`. Si el archivo ya existe, lo que implica que la tabla ya se ha creado previamente, se carga la tabla existente en un DataFrame llamados `tabla_promedios_tmax` y `tabla_promedios_tmin`.

In [6]:
# Ruta de la carpeta donde se almacenará los datos procesados
carpeta = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Prom_3_Dias_Anteriores/Datos/Temperatura/"

# Nombre de la tabla a crear o abrir que contendrá el promedio de la diferencia (pro - obs)
nombre_tabla1 = "tabla_promedios_tmax"
nombre_tabla2 = "tabla_promedios_tmin"

def crear_tabla_si_no_existe(carpeta, nombre_tabla1, nombre_tabla2):
    """
    Función que crea una tabla si esta no existe en la dirección proporcionada
    
    Parameters:
        carpeta (str): Ruta de la carpeta donde se guardará la tabla.
        nombre_tabla1, nombre_tabla2 (str): Nombre de la tabla a crear.
    Returns:
        None
    """
    # Comprobar si la carpeta existe, si no, la crea
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Ruta completa del archivo que contendrá la tabla
    ruta_archivo1 = os.path.join(carpeta, f"{nombre_tabla1}.csv")
    ruta_archivo2 = os.path.join(carpeta, f"{nombre_tabla2}.csv")

    # Si el archivo no existe, se crea la tabla vacía
    if not os.path.exists(ruta_archivo1) and not os.path.exists(ruta_archivo2):
        tabla_promedios_tmax = pro[0][["Estacion"]]
        tabla_promedios_tmin = pro[0][["Estacion"]]
    else:
        #Si el archivo existe, se cargan los datos
        tabla_promedios_tmax = pd.read_csv(ruta_archivo1)
        tabla_promedios_tmin = pd.read_csv(ruta_archivo2)
    return tabla_promedios_tmax, tabla_promedios_tmin
 
# Llamar a la función para crear o cargar la tabla y asignarla a variables
tabla_promedios_tmax, tabla_promedios_tmin = crear_tabla_si_no_existe(carpeta, nombre_tabla1, nombre_tabla2)

## Promedio entre Ayer, Anteayer y Antes de Anteayer

Se empieza por concatenar todos los DataFrames presentes en `tablas_diferencias` en un solo DataFrame llamado `tabla_concatenada`. Esta operación combina todas las tablas de diferencias en una única tabla, lo que facilita el procesamiento conjunto.

Se calcula el promedio de las diferencias de temperatura ("Diferencia_Tmax" y "Diferencia_Tmin") para cada estación. Los resultados se almacenan en DataFrames separados llamados `tabla_promedio_tmax` y `tabla_promedio_tmin`. Se utiliza el método groupby() para agrupar por la columna "Estacion" y luego se calcula el promedio utilizando mean(). Finalmente, se reinician los índices con reset_index().

Se obtiene la fecha actual en formato de texto y se almacena en la variable `text_hoy` con el formato "%Y%m%d". Luego, se renombran las columnas de los DataFrames `tabla_promedio_tmax` y `tabla_promedio_tmin` con la fecha actual como nombre de columna.

Se verifica si la columna con el nombre de la fecha actual ya existe en los DataFrames `tabla_promedios_tmax` y `tabla_promedios_tmin`. Si existe, se elimina la columna correspondiente utilizando el método drop().

Se fusionan los DataFrames `tabla_promedios_tmax` y `tabla_promedio_tmax` en uno solo, utilizando la columna `Estacion` como clave. Se realiza una fusión externa ("outer") para obtener solo las estaciones que coinciden. El mismo proceso se repite para los DataFrames `tabla_promedios_tmin` y `tabla_promedio_tmin`.

La tabla resultante, que ahora contiene los promedios de las diferencias junto con los datos históricos, se guarda en un archivo CSV en la ubicación especificada por `carpeta`.

In [7]:
from datetime import datetime, date, timedelta  # Importamos clases relacionadas con fechas y horas.

# Concatenar las tres tablas de tablas_diferencias en una sola tabla
tabla_concatenada = pd.concat(tablas_diferencias)

# Calcular el promedio agrupando por Estación
tabla_promedio_tmax = tabla_concatenada.groupby("Estacion")["Diferencia_Tmax"].mean().reset_index()
tabla_promedio_tmin = tabla_concatenada.groupby("Estacion")["Diferencia_Tmin"].mean().reset_index()

# Obtener la fecha actual en formato texto y renombrar la columna 'Resta' de tabla_promedio_tmax, tabla_promedio_tmin
text_hoy = (date.today()).strftime("%Y%m%d")
tabla_promedio_tmax = tabla_promedio_tmax.rename(columns={"Diferencia_Tmax": f"{text_hoy}_tmax"})
tabla_promedio_tmin = tabla_promedio_tmin.rename(columns={"Diferencia_Tmin": f"{text_hoy}_tmin"})

# Verificar si la columna ya existe en tabla_promedios_tmax, tabla_promedios_tmin
if f"{text_hoy}_tmax" in tabla_promedios_tmax.columns:
    # Si existe, eliminarla
    tabla_promedios_tmax.drop(columns=[f"{text_hoy}_tmax"], inplace=True)
    
if f"{text_hoy}_tmin" in tabla_promedios_tmin.columns:
    # Si existe la columna existe, eliminarla
    tabla_promedios_tmin.drop(columns=[f"{text_hoy}_tmin"], inplace=True)

# Fusionar tabla_promedios_tmax, tabla_promedios_tmin con tabla_promedio_tmax, tabla_promedio_tmin
tabla_promedios_tmax = tabla_promedios_tmax.merge(tabla_promedio_tmax, on="Estacion", how="outer")
tabla_promedios_tmin = tabla_promedios_tmin.merge(tabla_promedio_tmin, on="Estacion", how="outer")

# Guardar la tabla_promedios_tmax, tabla_promedios_tmin en un archivo CSV y TXT
tabla_promedios_tmax.to_csv(f"{carpeta}/tabla_promedios_tmax.csv", index=False)
tabla_promedios_tmax.to_csv(os.path.join(carpeta, "tabla_promedios_tmax.txt"), index=False)

tabla_promedios_tmin.to_csv(f"{carpeta}/tabla_promedios_tmin.csv", index=False)
tabla_promedios_tmin.to_csv(os.path.join(carpeta, "tabla_promedios_tmin.txt"), index=False)

En primer lugar, se obtiene la fecha actual y la fecha de mañana en un formato específico ("YYYYMMDD"). Luego, se construye una URL para un archivo CSV utilizando estas fechas y un directorio de modelo previamente establecido. A continuación, se verifica si el archivo CSV correspondiente a la fecha de mañana existe en la ubicación especificada. Si el archivo existe, se carga como un DataFrame de Pandas para su posterior procesamiento. En caso contrario, se muestra un mensaje indicando que el archivo para la fecha de mañana no está disponible en la ubicación predeterminada.

In [8]:
# Obtén la fecha de hoy y mañana en el formato deseado
text_hoy = date.today().strftime("%Y%m%d")
text_mañana = (date.today() + timedelta(days=1)).strftime("%Y%m%d")

# Construir el URL del archivo de temperatura pronosticada para mañana
url_file = f"{directorio_modelo}/{text_mañana}.temperatura.csv"

# Verificar si el archivo existe en la ubicación especificada
if os.path.exists(url_file):
    # Leer el archivo CSV como un DataFrame de Pandas
    file = pd.read_csv(url_file)

else:
    # Imprimir un mensaje en caso de que el archivo de temperatura pronosticada no exista
    print(f'Archivo para la fecha {text_mañana} no existe')

Se combina los datos de tres DataFrames: `file`, `tabla_promedio_tmax`, y `tabla_promedio_tmin`, mediante operaciones de fusión basadas en la columna "Estacion". Luego, se realizan cálculos para obtener las temperaturas máximas y mínimas actualizadas. Se suman las columnas "Tmax" y "Tmin" del DataFrame `Pronostico` con las columnas correspondientes a la fecha actual (`text_hoy`) en los DataFrames fusionados.

Después de calcular las diferencias, se seleccionan solo las columnas necesarias en el DataFrame `Pronostico`, que son "Estacion", "Tmax" y "Tmin". Finalmente, los resultados se guardan en dos archivos: uno en formato CSV y otro en formato TXT, con la fecha de mañana incluida en el nombre de archivo.

In [13]:
# Unir el pronostico de mañana con la tabla del promedio (pro - obs) de los 3 días previos
Pronostico = file.merge(tabla_promedio_tmax, on="Estacion", how="left")
Pronostico = Pronostico.merge(tabla_promedio_tmin, on="Estacion", how="left")

# Restarle al pronostico de mañana el promedio de los 3 días previos
Pronostico["Tmax"] = Pronostico["Tmax"] - Pronostico[f"{text_hoy}_tmax"]
Pronostico["Tmin"] = Pronostico["Tmin"] - Pronostico[f"{text_hoy}_tmin"]

# Seleccionar solo las columnas necesarias
Pronostico = Pronostico[["Estacion", "Tmax", "Tmin"]]

# Guarda el DataFrame del ajuste del pronostico de mañana en archivos CSV y TXT
Pronostico.to_csv(f"{carpeta}/{text_mañana}.pron_temperatura.csv", index=False)
Pronostico.to_csv(os.path.join(carpeta, f"{text_mañana}.pron_temperatura.txt"), index=False)